# Day 1

In [46]:
with open('day1/input.txt', 'r') as f:
    arr = f.read().splitlines()

In [63]:
arr = [int(x) for x in arr]

def two_sum(arr, x):
    arr.sort()
    l = 0
    h = len(arr) - 1
    while l < h:
        if arr[l] + arr[h] == x:
            return arr[l] * arr[h]
        elif arr[l] + arr[h] < x:
            l += 1
        else:
            h -= 1
    return -1


In [65]:
def three_sum(arr, x):
    arr.sort()
    l = 0
    h = len(arr) - 1
    for y in arr:
        tmp = two_sum(arr, x - y)
        if tmp != -1:
            return tmp * y

In [67]:
two_sum(arr, 2020)

960075

In [68]:
three_sum(arr, 2020)

212900130

# Day 2

In [22]:
import re

# part 1
n = 0
with open('day2/input.txt', 'r') as f:
    for line in f:
        l, h, letter, pwd = re.split('[^A-Za-z0-9]+', line)[:-1]
        l, h = int(l), int(h)
        
        if len(re.findall(letter, pwd)) in range(l, h + 1):
            n += 1
            
print(f"For Part 1 there are {n} correct passwords")

There are 625 correct passwords


In [43]:
# part 2
n = 0
with open('day2/input.txt', 'r') as f:
    for line in f:
        l, h, letter, pwd = re.split('[^A-Za-z0-9]+', line)[:-1]
        l, h = int(l), int(h)
        
        try:
            l_letter = pwd[l-1]
            h_letter = pwd[h-1]
        except IndexError:
            continue
        
        if (l_letter == letter) != (h_letter == letter):
            n += 1

In [44]:
print(f"For Part 2 there are {n} correct passwords")

For Part 2 there are 391 correct passwords


# Day 3

In [54]:
import numpy as np

with open('day3/input.txt', 'r') as f:
    arr = f.read().splitlines()

In [45]:
def get_trees(arr, right, down):
    x = 0; y = 0
    xmax = len(arr[0]) 
    ymax = len(arr) - 1
    
    n_trees = 0
    while True:
        x  = (x + right) % xmax
        y += down

        if y > ymax:
            break
        if arr[y][x] == "#":
            n_trees += 1
            
    return n_trees


In [47]:
right = 3
down = 1
n_trees = get_trees(arr, right, down)
print(f"Following right {right} and down {down} we hit {n_trees} trees")

Following right 3 and down 1 we hit 193 trees


In [55]:
slopes = zip([1, 3, 5, 7, 1], [1, 1, 1, 1, 2])
trees = [get_trees(arr, x, y) for x, y in slopes]
tree_product = np.prod(trees)

In [58]:
print(f"Part two answer: {tree_product}")

Part two answer: 1355323200


# Day 4

In [16]:
import re
import numpy as np

In [244]:
# functions
def parse_line(entry):  
    # split on whitespace or newlines
    full = re.split('[\s\n]', entry)
    
    # remove empty strings
    full = [s for s in full if s]
    
    # break into keys and values
    keyvals = np.transpose([x.split(':') for x in full])
    keys, values = keyvals[0], keyvals[1]
    
    # return dict
    return dict(zip(keys, values))

def has_req_fields(entry):
    req_fields = set(['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid'])
    missing = req_fields - set(entry.keys())
    return len(missing) == 0

def validate_field(field, value):
    # expects field (str) and value (str), return True if valid, False otherwise
    if field == 'byr':
        if not value.isdigit():
            return False
        return int(value) in range(1920, 2003)

    if field == 'iyr':
        if not value.isdigit():
            return False
        return int(value) in range(2010, 2021)

    if field == 'eyr':
        if not value.isdigit():
            return False
        return int(value) in range(2020, 2031)

    if field == 'hgt':
        if not re.fullmatch('\d+(cm|in)', value):
            return False
        digit = int(re.findall('\d+', value)[0])
        unit = re.findall('cm|in', value)[0]
        if unit == 'cm':
            return digit in range(150, 194)
        if unit == 'in':
            return digit in range(59, 77)

    if field == 'hcl':
        return bool(re.fullmatch('#[a-f0-9]{6}', value))

    if field == 'ecl':
        return value in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']

    if field == 'pid':
        return bool(re.fullmatch('\d{9}', value))
    
def all_fields_correct(entry):
    for field in entry:
        # ignore country id field
        if field == 'cid':
            continue
        if not validate_field(field, entry[field]):
            return False
    return True

In [245]:
# read data
with open('day4/input.txt', 'r') as f:
    raw = f.read().split('\n\n')
entries = [parse_line(x) for x in raw]

# part 1
n_valid = 0
for entry in entries:
    if has_req_fields(entry):
        n_valid += 1

print(f"Part 1: {n_valid} valid fields")

Part 1: 247 valid fields


In [246]:
# part 2
n_valid = 0
for entry in entries:
    if has_req_fields(entry) and all_fields_correct(entry):
        n_valid += 1
        
print(f"Part 2: {n_valid} valid fields")

Part 2: 145 valid fields
